In [42]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [2]:
FEATURES = [
 "doc_top_diag", 
 "doc_gen_diag", 
 "doc_top_oper", 
 "doc_gen_oper", 
]

In [ ]:
clinic_ids = [i['id'] for i in estims_data]
#estims_data.pop(clinic_ids.index('0'))
#print sorted(clinic_ids)
#print sorted([i['id'] for i in cancer_data])

In [ ]:
len(cancer_data[9]['doctors'])

In [3]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        ids_diad = [i['id'] for i in doc['procedures']['diagnostics'] if i['top']=='1']
        ids_oper = [i['id'] for i in doc['procedures']['operations'] if i['top']=='1']
        X += [[
            len(list(set(ids_diad))),
            len(doc['procedures']['diagnostics']),
            len(list(set(ids_oper))),
            len(doc['procedures']['operations']),
        ]]
    y=[int(clinic_estim['experience'])] * len(all_doc)
    return [X, y]

In [4]:
def normale(mat):
    for i in [0,1,2,3]:
        mass = [k[i] for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [5]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [6]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print "%.3f" % np.mean(abs(regr.predict(train_X) - train_y))
        print "%.3f" % np.mean(abs(regr.predict(test_X) - test_y))
        print "%.3f" % (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100)
        print "%.3f" % regr.score(train_X, train_y)
        print "%.3f" % regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    for i in regr.coef_:
        print "%.3f" % i
    print "%.3f" % regr.intercept_
    return regr

In [40]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

normale(X_all)

In [ ]:
X, y = get_X_y_from(cancer_data)

In [41]:
len(X_all)

64

64

In [39]:
for i in range(1,10):
    print i
    regr = process_with(X_all, y_all, short=True, ts=i/10.0)
    print

1
Total: 54, train: 48, test: 6
0.614
0.625
7.356
0.414
-0.069
-1.956
-1.345
0.592
1.162
9.195

2
Total: 54, train: 43, test: 11
0.569
0.782
9.058
0.496
-0.786
-1.579
-2.457
0.831
1.519
9.283

3
Total: 54, train: 37, test: 17
0.513
0.843
9.748
0.585
-0.524
-1.788
-2.317
0.810
1.385
9.407

4
Total: 54, train: 32, test: 22
0.553
0.705
8.125
0.561
-0.247
-1.805
-2.299
0.819
1.436
9.376

5
Total: 54, train: 27, test: 27
0.617
0.611
7.083
0.519
0.020
-1.960
-1.947
0.803
1.241
9.392

6
Total: 54, train: 21, test: 33
0.567
0.646
7.451
0.566
0.030
-2.820
-1.104
0.827
0.960
9.471

7
Total: 54, train: 16, test: 38
0.593
0.658
7.559
0.554
0.112
-2.985
-1.199
0.850
1.846
9.046

8
Total: 54, train: 10, test: 44
0.696
0.728
8.476
0.412
0.058
-4.084
2.232
-0.223
0.692
9.042

9
Total: 54, train: 5, test: 49
0.000
7.356
86.442
1.000
-116.078
8.063
-52.222
18.069
18.065
12.009

1
Total: 54, train: 48, test: 6
0.614
0.625
7.356
0.414
-0.069
-1.956
-1.345
0.592
1.162
9.195

2
Total: 54, train: 43, test: 1

In [ ]:
regr = process_with(X_all, y_all, info=True, ts=0.1)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y)):
    print "Real: %f \t Predicted: %f" %(y[i], regr.predict(X[i]))

In [27]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])
    #print i[0]

Features sorted by their score:
8.92592592593
4
-0.550100 	 doc_top_diag
-0.846200 	 doc_gen_diag
0.000000 	 doc_top_oper
2.333600 	 doc_gen_oper
Features sorted by their score:
8.92592592593
4
-0.550100 	 doc_top_diag
-0.846200 	 doc_gen_diag
0.000000 	 doc_top_oper
2.333600 	 doc_gen_oper


In [ ]:
X_all

In [38]:
ind = X_all.index([0,0,0,0])
X_all.pop(ind)
y_all.pop(ind)

ValueError: [0, 0, 0, 0] is not in list

ValueError: [0, 0, 0, 0] is not in list

In [ ]:
coef = np.array([-0.7824,0.2034,0.0096,0.077,8.4162])
for x in X:
    print sum(x*coef[:-1])+coef[-1]


In [ ]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
plt.plot([i[22] for i in X])
plt.show()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)